### Heatmap of the top fGWAS results

In [6]:
setwd("../private_output/Fgwas/")

In [2]:
library(plotrix)
suppressPackageStartupMessages(library(gplots))

In [7]:
res = read.table('~/Family1070/private_output/fgwas_analysis/HR/pipeline_out/Final_Model/Final_Model.bfs', header=T)
seg = read.table('~/Family1070/private_output/fgwas_analysis/HR/pipeline_out/Final_Model/Final_Model.segbfs', header=T)
res = merge(res,seg[,c('chunk', 'PPA')], by="chunk" )
res$PPA = res$PPA.x *res$PPA.y
res = res[order(res$PPA, decreasing=T),]

*Choose a cutoff of  PPA, then re-adjust the PPA by multipling by 1 if the Locus is known

In [8]:
seg = res[res$PPA >0.3, ]

In [11]:
gwas = "/publicdata/gwas_summary_stats_20180124/denHoed/rsid_pval.txt"
p = paste( "grep -E '", paste(seg$id, collapse="|"), "' ", gwas ," > HR_selected_variants.tsv", sep="")

In [12]:
system(p)

In [13]:
r = read.table("HR_selected_variants.tsv")

In [16]:
seg = merge(seg, r, by.x="id", by.y="V1")
colnames(seg)[ncol(seg)] = "P_value"

In [17]:
writeLines(as.character(seg$id), "TOP_rs_variants_HR")

In [18]:
system("module load cardips")

command1 = "vcftools --vcf /publicdata/haploreg_v4.0_20151021/haploreg_v4.0_20151021.vcf --snps TOP_rs_variants_HR --recode --recode-INFO-all --out HR"
### double escape to use in R
command2 = paste("bcftools query -f ", "'%ID\\t%INFO/DBSNP\\t%INFO/NEAREST_GENCODE\\n' ",
                    "HR.recode.vcf > HR_haploreg.tab", sep="")
           
system(command1)
system(command2)

In [19]:
a = read.table("HR_haploreg.tab")
colnames(a)=c('id', 'dbSNP', 'gencode')
seg = merge(seg, a, by=1)

In [20]:
seg = seg[order(seg$PPA, decreasing=T),]
write.csv(seg, 'fgwas_results_HR.csv', row.names=FALSE)

In [43]:
seg = subset(seg, dbSNP!="NSM")

In [46]:
se = seg[,c('NKX25_ASE','NKX25','H3K27AC','ATAC')]

In [47]:
mat = seg[rowSums(se)>0 & seg$P_value<1e-3 , c('id',  'PPA','NKX25_ASE','NKX25','H3K27AC','ATAC')]

In [49]:
mat$GWAS_P = mat$id %in% seg$id[seg$P_value<1e-8]
#mat$GWAS_other      = mat$id %in% seg$id[seg$P_value<1e-4]

mat2 = mat[,2:length(mat)]
rownames(mat2)=mat$id

lab = mat2
lab[, 2:length(mat2)]<-NA
lab[,1]<-round(lab[,1],2)

suppressPackageStartupMessages(library(RColorBrewer))
cols = brewer.pal(9, "Reds")[2:8]

breaks=seq(0.5, 1.1, by=0.1) # cutoff 0.5
breaks=seq(0.3, 1.1, by=0.1) # cutoff 0.3

my_palette = c(colorRampPalette(cols)(length(breaks)-2), "deepskyblue3")

In [50]:
mat2[mat2==0]<- NA
mat2[mat2==TRUE]<- 1
mat2[mat2==1]<- 2

In [57]:
pdf('Table_snps_fgwas_heart_rate.pdf')
heatmap.2(as.matrix(mat2), 
         Colv=F,Rowv=F, srtRow=0, srtCol=45, offsetRow=-1, offsetCol=-0.5,dendrogram="none",
          keysize=1, margins =c(25,25), trace="none",key.title="OR", tracecol=NA,
          cexRow=1, cexCol=1, na.color="white", col=my_palette, breaks=breaks, cellnote=lab, notecex=0.8)
dev.off()

png 
  2

In [52]:
m = seg[rowSums(se)>0 & seg$P_value<1e-3 ,]

In [64]:
m = seg[rowSums(se)>0,]

In [65]:
dim(m)

[1] 11 21